# EDA
This file will take some time loking at the data and trying to figure out what can be useful. There are quite a few tables so it will be helpful to get familiar with exactly what information is available.  
I use a custom class I defined called DataLoader ot deal with some of the tedium of connecting to the database and making common queries.

In [11]:
import psycopg2
import pandas as pd
import seaborn as sns
import statsmodels as sm
from matplotlib import pyplot as plt

from DataLoader import DataLoader

In [10]:
dl = DataLoader.DataLoader('config.json') 

There are quite a few table sin the database and a lot of them don't contain much information. I grouped them into a couple categories based on how useful they would be for predicting match outcomes from ingame data, the main purpose of this project.

In [3]:
dl.get_table(dl.tablename_query, index=None)

,table_name
0,maps
1,users
2,files
3,objects
4,civilization_bonuses
5,alembic_version
6,player_colors
7,object_instance_states
8,tournaments
9,civilizations


## Miscellaneous Tables
These tables contain information that might be useful to someone but not really me, at least for now. I might use some more of the information about series and tournaments for other things but for now I will leave them aside.  
users, files, people, hc, events, rounds, series, and series_metadata

## Reference tables
There are quite a few tables that are present just to convert between numeric identifiers and other datatypes, primarily strings, so I won't spend much time with those. The tables maps, civbonus, color, civ, mapsize, eventmap, datasets, gametype, tech, difficulties, mapreveal, speed, startres, startage, victory_conditions, terrain, versions, actions, resources, formation_types, objects, tournaments and ladder all fall into this category.


In [4]:
dl.color

,name,hex
id,,
0,Blue,#0000FF
1,Red,#FF0000
2,Green,#00FF00
3,Yellow,#FFFF00
5,Purple,#FF00FF
6,Gray,#2F2F2F
7,Orange,#FF8201
4,Teal,#00FFFF


In [5]:
dl.datasets

,name,short
id,,
1,Wololo Kingdoms,wk
0,The Conquerors,aoc
100,Definitive Edition,de
200,Age of Kings,aok
7,Realms,realms
2,Portuguese Civilization Mod III,pcm


In [6]:
dl.civbonus.head()

,civilization_id,dataset_id,type,description
id,,,,
1,1,1,civ,Town Centers cost -50% wood in Castle Age
2,1,1,civ,Foot Archers (except Skirmishers) have +1/+2 r...
3,1,1,civ,Shepherds work 25% faster
4,1,1,team,Archery Ranges work 20% faster
5,2,1,civ,Foragers work 25% faster


## Match Data
These are the tables that I will primarily be working with. The reference tables will be useful for translating some things into something more understandable than the numbers while I am getting more familiar with the way that they are encoded but this is where I will spend the rest of my time in this file.

In [129]:
dl.get_table(dl.player_query, limit=1000)

,match_id,name,number,color_id,platform_id,user_id,team_id,dataset_id,civilization_id,start_x,...,explored_percent,research_count,research_percent,society_score,total_wonders,total_castles,total_relics,villager_high,user_name,cheater
0,561810,mercurious,2,0,voobly,123146863,1,0,8,104,...,86.0,44.0,43.0,390.0,0.0,3.0,0.0,115.0,mercurious,None
1,561810,danilo1710,1,7,voobly,123397123,0,0,11,22,...,59.0,28.0,28.0,260.0,0.0,2.0,0.0,76.0,danilo1710,None
2,561811,kestafa,2,1,voobly,124980415,1,1,14,89,...,60.0,15.0,9.0,0.0,0.0,0.0,0.0,50.0,kestafa,None
3,561811,scorpion1980,1,0,voobly,125336371,0,1,5,11,...,42.0,9.0,5.0,0.0,0.0,0.0,0.0,55.0,scorpion1980,None
4,561812,IamCerro,1,5,voobly,123280315,0,1,3,97,...,48.0,13.0,9.0,0.0,0.0,0.0,0.0,44.0,IamCerro,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,561876,GodKingEmperor,5,7,voobly,124474834,0,0,7,139,...,100.0,34.0,32.0,260.0,0.0,2.0,0.0,138.0,GodKingEmperor,None
996,153878,HSHS,3,4,vooblycn,124625553,1,0,5,20,...,39.0,34.0,34.0,260.0,0.0,2.0,0.0,143.0,None,None
997,561876,NoFear1907,6,0,voobly,124577545,0,0,13,192,...,100.0,35.0,35.0,130.0,0.0,1.0,1.0,169.0,NoFear1907,None
998,533532,BlackRock29,3,3,voobly,125351152,1,0,10,151,...,100.0,52.0,49.0,520.0,0.0,4.0,2.0,141.0,BlackRock29,None


In [7]:
dl.get_table('SELECT COUNT(*) FROM matches')

,count
0,2157841


In [8]:
dl.get_table('SELECT COUNT(DISTINCT match_id) FROM timeseries')

,count
0,14223


The number of matches which have basic info is much higher than those with information of things happening in game so I will subset the tables to just those. 

In [118]:
match_columns = 'id,dataset_id,ladder_id,rated,map_name,played,duration,completed,postgame,map_reveal_choice_id,speed_id,diplomacy_type,starting_resourceS_id,starting_age_id,victory_condition_id,multiqueue,map_tiles,version_id,winning_team_id'

In [ ]:
df = dl.get_table('SELECT {0} FROM matches WHERE id IN (SELECT DISTINCT match_id FROM timeseries)'.format(match_columns))

In [ ]:
df.ladder_id.value_counts()

In [ ]:
df.map_name.value_counts().head(30)

Arabia is the most standard and while there are differences across versions it should be reasonable to consider each of the scripts that generate Arabia as the same. Because there are not a lot of matches of other maps I will not consider those for now.

In [119]:
df = dl.get_table("SELECT {0} FROM matches WHERE id IN (SELECT DISTINCT match_id FROM timeseries) AND map_name SIMILAR TO '%(Arabia|G\.? ?A)%'".format(match_columns))

In [7]:
df.map_name.value_counts()[:10]

Arabia                4685
HC2_Arabia            3974
ECL_Arabia            1246
KotD2 - Arabia         838
Green Arabia           166
HC2_Arabia_v2          130
KotD2_Arabia_v2        125
KotD2 - Arabia - 2     101
NAC_Arabia              87
!! G. A voobly          81
Name: map_name, dtype: int64

In [26]:
df.columns

Index(['id', 'dataset_id', 'ladder_id', 'rated', 'map_name', 'played',
       'duration', 'completed', 'postgame', 'map_reveal_choice_id', 'speed_id',
       'diplomacy_type', 'starting_resources_id', 'starting_age_id',
       'victory_condition_id', 'multiqueue', 'map_tiles', 'version_id',
       'objects', 'winning_team_id'],
      dtype='object')

In [65]:
ts = dl.get_table("SELECT * FROM timeseries WHERE match_id IN (SELECT id FROM matches WHERE matches.map_name SIMILAR TO '%(Arabia|G\.? ?A)%')")

In [66]:
ts.isnull().sum()

id                               0
match_id                         0
player_number                    0
timestamp                        0
population                       0
military                         0
percent_explored                 0
headroom                         0
food                             0
wood                             0
stone                            0
gold                             0
relic_gold                       0
relics_captured                  0
total_food                       0
total_gold                       0
total_stone                      0
total_wood                       0
trade_profit                     0
tribute_received                 0
tribute_sent                     0
total_housed_time          3809228
total_popcapped_time       3809228
value_current_buildings    3809228
value_current_units        3809228
value_lost_buildings       3809228
value_lost_units           3809228
value_objects_destroyed    3809228
value_spent_objects 

In [121]:
ts.columns

Index(['id', 'population', 'military', 'percent_explored', 'headroom', 'food',
       'wood', 'stone', 'gold', 'relic_gold', 'relics_captured', 'total_food',
       'total_gold', 'total_stone', 'total_wood', 'trade_profit',
       'tribute_received', 'tribute_sent', 'total_housed_time',
       'total_popcapped_time', 'value_current_buildings',
       'value_current_units', 'value_lost_buildings', 'value_lost_units',
       'value_objects_destroyed', 'value_spent_objects',
       'value_spent_research', 'deaths', 'kills', 'razes', 'buildings_lost',
       'converted', 'economy_score', 'military_score', 'society_score',
       'technology_score', 'total_score', 'trained', 'value_buildings_razed',
       'value_units_killed', 'hit_points_killed', 'hit_points_razed',
       'military_high', 'villager_high'],
      dtype='object')

In [89]:
ts = ts.drop_duplicates(subset=['match_id','timestamp','player_number']) # dropping a problematic duplicate
ts = ts.set_index(['match_id','timestamp','player_number'])

In [90]:
ts.unstack()

id                                       \
player_number                       1            2   3   4   5   6   7   8   
match_id timestamp                                                           
38       00:00:03.358000  172880412.0  172880413.0 NaN NaN NaN NaN NaN NaN   
         00:00:33.365000  172880414.0  172880415.0 NaN NaN NaN NaN NaN NaN   
         00:01:03.395000  172880416.0  172880417.0 NaN NaN NaN NaN NaN NaN   
         00:01:33.425000  172880418.0  172880419.0 NaN NaN NaN NaN NaN NaN   
         00:02:03.455000  172880420.0  172880421.0 NaN NaN NaN NaN NaN NaN   
...                               ...          ...  ..  ..  ..  ..  ..  ..   
1998651  00:41:10.018000  173341436.0  173341437.0 NaN NaN NaN NaN NaN NaN   
         00:41:40.986000  173341438.0  173341439.0 NaN NaN NaN NaN NaN NaN   
         00:42:11.814000  173341440.0  173341441.0 NaN NaN NaN NaN NaN NaN   
         00:42:42.082000  173341442.0  173341443.0 NaN NaN NaN NaN NaN NaN   
         00:42:54.859000  173341444.0  173341445.0 NaN NaN NaN NaN NaN NaN   

                         population         ... military_high      \
player_number                     1      2  ...             7   8   
match_id timestamp                          ...                     
38       00:00:03.358000        4.0    4.0  ...           NaN NaN   
         00:00:33.365000        5.0    5.0  ...           NaN NaN   
         00:01:03.395000        6.0    6.0  ...           NaN NaN   
         00:01:33.425000        7.0    7.0  ...           NaN NaN   
         00:02:03.455000        8.0    8.0  ...           NaN NaN   
...                             ...    ...  ...           ...  ..   
1998651  00:41:10.018000      147.0  108.0  ...           NaN NaN   
         00:41:40.986000      154.0  106.0  ...           NaN NaN   
         00:42:11.814000      156.0  102.0  ...           NaN NaN   
         00:42:42.082000      164.0  109.0  ...           NaN NaN   
         00:42:54.859000      161.0  104.0  ...           NaN NaN   

                         villager_high                                
player_number                        1     2   3   4   5   6   7   8  
match_id timestamp                                                    
38       00:00:03.358000           NaN   NaN NaN NaN NaN NaN NaN NaN  
         00:00:33.365000           NaN   NaN NaN NaN NaN NaN NaN NaN  
         00:01:03.395000           NaN   NaN NaN NaN NaN NaN NaN NaN  
         00:01:33.425000           NaN   NaN NaN NaN NaN NaN NaN NaN  
         00:02:03.455000           NaN   NaN NaN NaN NaN NaN NaN NaN  
...                                ...   ...  ..  ..  ..  ..  ..  ..  
1998651  00:41:10.018000         115.0  75.0 NaN NaN NaN NaN NaN NaN  
         00:41:40.986000         120.0  75.0 NaN NaN NaN NaN NaN NaN  
         00:42:11.814000         126.0  78.0 NaN NaN NaN NaN NaN NaN  
         00:42:42.082000         126.0  79.0 NaN NaN NaN NaN NaN NaN  
         00:42:54.859000         126.0  80.0 NaN NaN NaN NaN NaN NaN  

[2748222 rows x 352 columns]

In [68]:
ts.player_number.value_counts()

2    2748223
1    2748223
3     127578
4     127360
5      75007
6      74027
7      42002
8      41552
Name: player_number, dtype: int64

In [70]:
ts.duplicated().sum()

0

In [73]:
temp.unstack()

ValueError: Index contains duplicate entries, cannot reshape

In [88]:
ts[ts.duplicated(keep=False)]

,id,match_id,player_number,timestamp,population,military,percent_explored,headroom,food,wood,...,society_score,technology_score,total_score,trained,value_buildings_razed,value_units_killed,hit_points_killed,hit_points_razed,military_high,villager_high


In [77]:
ts.duplicated(subset=['player_number'])

0          False
1          False
2           True
3           True
4           True
           ...  
5983967     True
5983968     True
5983969     True
5983970     True
5983971     True
Length: 5983972, dtype: bool

In [79]:
ts.duplicated(keep=False)

Index(['id', 'match_id', 'player_number', 'timestamp', 'population',
       'military', 'percent_explored', 'headroom', 'food', 'wood', 'stone',
       'gold', 'relic_gold', 'relics_captured', 'total_food', 'total_gold',
       'total_stone', 'total_wood', 'trade_profit', 'tribute_received',
       'tribute_sent', 'total_housed_time', 'total_popcapped_time',
       'value_current_buildings', 'value_current_units',
       'value_lost_buildings', 'value_lost_units', 'value_objects_destroyed',
       'value_spent_objects', 'value_spent_research', 'deaths', 'kills',
       'razes', 'buildings_lost', 'converted', 'economy_score',
       'military_score', 'society_score', 'technology_score', 'total_score',
       'trained', 'value_buildings_razed', 'value_units_killed',
       'hit_points_killed', 'hit_points_razed', 'military_high',
       'villager_high'],
      dtype='object')

In [92]:
test = dl.get_table('SELECT * FROM matches LIMIT 10')

In [93]:
test.columns

Index(['id', 'hash', 'series_id', 'tournament_id', 'event_id', 'version',
       'minor_version', 'dataset_id', 'dataset_version', 'platform_id',
       'ladder_id', 'rated', 'winning_team_id', 'builtin_map_id',
       'map_size_id', 'map_name', 'event_map_id', 'rms_zr', 'rms_custom',
       'rms_seed', 'guard_state', 'fixed_positions', 'direct_placement',
       'effect_quantity', 'played', 'added', 'platform_match_id', 'duration',
       'completed', 'restored', 'postgame', 'type_id', 'difficulty_id',
       'population_limit', 'map_reveal_choice_id', 'cheats', 'speed_id',
       'lock_teams', 'mirror', 'diplomacy_type', 'team_size',
       'starting_resources_id', 'starting_age_id', 'victory_condition_id',
       'team_together', 'all_technologies', 'lock_speed', 'multiqueue',
       'lobby_name', 'treaty_length', 'build', 'map_tiles', 'has_playback',
       'game_version', 'save_version', 'version_id', 'hc',
       'starting_palisades', 'starting_town_centers', 'starting_walls',
  

In [94]:
test.winning_team_id

0    0
1    0
2    6
3    0
4    0
5    0
6    0
7    1
8    1
9    0
Name: winning_team_id, dtype: int64

In [95]:
ts.columns

Index(['id', 'population', 'military', 'percent_explored', 'headroom', 'food',
       'wood', 'stone', 'gold', 'relic_gold', 'relics_captured', 'total_food',
       'total_gold', 'total_stone', 'total_wood', 'trade_profit',
       'tribute_received', 'tribute_sent', 'total_housed_time',
       'total_popcapped_time', 'value_current_buildings',
       'value_current_units', 'value_lost_buildings', 'value_lost_units',
       'value_objects_destroyed', 'value_spent_objects',
       'value_spent_research', 'deaths', 'kills', 'razes', 'buildings_lost',
       'converted', 'economy_score', 'military_score', 'society_score',
       'technology_score', 'total_score', 'trained', 'value_buildings_razed',
       'value_units_killed', 'hit_points_killed', 'hit_points_razed',
       'military_high', 'villager_high'],
      dtype='object')

In [99]:
df.winning_team_id.value_counts()

0    6061
1    5902
Name: winning_team_id, dtype: int64

In [100]:
df.columns

Index(['id', 'dataset_id', 'ladder_id', 'rated', 'map_name', 'played',
       'duration', 'completed', 'postgame', 'map_reveal_choice_id', 'speed_id',
       'diplomacy_type', 'starting_resources_id', 'starting_age_id',
       'victory_condition_id', 'multiqueue', 'map_tiles', 'version_id',
       'objects', 'winning_team_id'],
      dtype='object')

In [112]:
dl.get_table('SELECT diplomacy_type,winning_team_id,COUNT(*) FROM matches GROUP BY diplomacy_type,winning_team_id')

,diplomacy_type,winning_team_id,count
0,1v1,0.0,645399
1,1v1,1.0,624251
2,1v1,NaN,278
3,FFA,0.0,7216
4,FFA,1.0,5788
5,FFA,2.0,5247
6,FFA,3.0,1981
7,FFA,4.0,1425
8,FFA,5.0,1156
9,FFA,6.0,931


In [104]:
import re

In [120]:
df.diplomacy_type.value_counts()

1v1      10524
TG        1438
Other        1
Name: diplomacy_type, dtype: int64

In [123]:
dl.get_table(dl.tablename_query)

,table_name
0,maps
1,users
2,files
3,objects
4,civilization_bonuses
5,alembic_version
6,player_colors
7,object_instance_states
8,tournaments
9,civilizations


In [127]:
dl.get_table(dl.team_query, limit=10)

,team_id,match_id,winner
0,0,1,False
1,1,1,True
2,0,2,False
3,1,2,True
4,0,3,True
5,1,3,False
6,0,4,False
7,1,4,True
8,0,5,False
9,1,5,True


In [128]:
dl.get_table(dl.player_query, limit=10)

,match_id,name,number,color_id,platform_id,user_id,team_id,dataset_id,civilization_id,start_x,...,explored_percent,research_count,research_percent,society_score,total_wonders,total_castles,total_relics,villager_high,user_name,cheater
0,561810,mercurious,2,0,voobly,123146863,1,0,8,104,...,86,44,43,390,0,3,0,115,mercurious,None
1,561810,danilo1710,1,7,voobly,123397123,0,0,11,22,...,59,28,28,260,0,2,0,76,danilo1710,None
2,561811,kestafa,2,1,voobly,124980415,1,1,14,89,...,60,15,9,0,0,0,0,50,kestafa,None
3,561811,scorpion1980,1,0,voobly,125336371,0,1,5,11,...,42,9,5,0,0,0,0,55,scorpion1980,None
4,561812,IamCerro,1,5,voobly,123280315,0,1,3,97,...,48,13,9,0,0,0,0,44,IamCerro,None
5,533461,182_Igoboom_Arg,3,4,voobly,124514874,1,0,16,113,...,100,7,7,0,0,0,0,30,182_Igoboom_Arg,None
6,533461,StorM_Disaster,4,5,voobly,124393327,1,0,13,58,...,100,9,9,0,0,0,0,34,StorM_Disaster,None
7,533461,Oderich,5,6,voobly,124752372,1,0,7,19,...,100,6,5,0,0,0,0,36,Oderich,None
8,533461,RageDesert_,8,1,voobly,124698202,1,0,2,201,...,100,8,8,0,0,0,0,35,RageDesert_,None
9,533461,_ExeCutioner,1,0,voobly,124410792,0,0,11,164,...,100,10,10,0,0,0,0,40,_ExeCutioner,None
